In [1]:
import pickle
import copy
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [2]:
benchs = ['lb', 'mmlu', 'helm_lite', 'alpaca']
number_item = 100
method = 'anchor-irt'
tinyBenchmarks = {}

In [3]:
for bench in benchs:
    with open(f'results/samples_{bench}_iterations-5.pickle', 'rb') as handle:
        sample_data = pickle.load(handle)

    scenarios = list(sample_data['scenarios_position'].keys())

    with open(f'results/results_{bench}_split-iid_iterations-5.pickle', 'rb') as handle:
        results = pickle.load(handle)
        avg_error = np.mean([np.mean([results[it][number_item][method+"_naive"][scenario] for it in results.keys()],axis=0) for scenario in scenarios], axis=0)
    
    print(bench,np.min(avg_error),avg_error)
    best_it = np.argmin(avg_error)
    
    optimal_lambdas = {}
    for scenario in scenarios:
        optimal_lambdas[scenario] = sample_data['opt_lambds'][method+"_gpirt"][scenario][number_item]
    
    tinyBenchmarks[bench] = {'seen_examples':sample_data['seen_items'][method][number_item][best_it],
                             'examples_weights':sample_data['item_weights'][method][number_item][best_it],
                             'irt_parameters':{"A":sample_data["A"], "B":sample_data["B"]},
                             'scenarios_position':sample_data['scenarios_position'],
                             'subscenarios_position':sample_data['subscenarios_position'],
                             'optimal_lambdas':optimal_lambdas}

lb 0.01906167653388267 [0.01906168 0.01998235 0.02033831 0.0202809  0.02015149]
mmlu 0.024398970156813124 [0.02728616 0.02829303 0.03106703 0.02439897 0.03167772]
helm_lite 0.0242250825876071 [0.02748594 0.02689979 0.02783222 0.02589602 0.02422508]
alpaca 0.01119042421102733 [0.0116694  0.01212778 0.01119042 0.01195137 0.01160708]


In [4]:
with open('results/tinyBenchmarks.pkl', 'wb') as f:
    pickle.dump(tinyBenchmarks, f)

In [11]:
!pip install git+https://github.com/felipemaiapolo/tinyBenchmarks

  Cloning https://github.com/felipemaiapolo/tinyBenchmarks to /tmp/pip-req-build-2p36arzo
  Running command git clone --filter=blob:none --quiet https://github.com/felipemaiapolo/tinyBenchmarks /tmp/pip-req-build-2p36arzo
